In [1]:
import numpy as np
import pandas as pd

In [3]:
np.random.seed(42)
df = pd.read_csv('clean_data.csv')
imdb_df = pd.read_csv('clean_imdb.csv')
pdf = df.reindex(np.random.permutation(df.index))
np.random.seed(42)
imdb_pdf = imdb_df.reindex(np.random.permutation(imdb_df.index))

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
X_train, X_test, y_train, y_test = imdb_pdf['clean_text'], pdf['clean_text'], imdb_pdf['class'], pdf['class']

In [11]:
vec = CountVectorizer(ngram_range=(1,2))
X_train_enc = vec.fit_transform(X_train)
X_test_enc = vec.transform(X_test)

In [12]:
X_train_enc.shape

(50000, 3232842)

In [13]:
from vecstack import stacking

lr = LogisticRegression(random_state=42)
svc = LinearSVC(random_state=42)
models = [lr, svc, SGDClassifier(random_state=42, penalty='elasticnet', l1_ratio=.1, learning_rate='optimal')]

In [14]:
S_train, S_test = stacking(models,
                          X_train_enc, y_train, X_test_enc,
                           verbose=2, random_state=42, shuffle=True, regression=False,
                          n_folds=5, metric=accuracy_score, mode='oof_pred_bag')

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LogisticRegression]


/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  0:  [0.90630000]
    fold  1:  [0.90700000]
    fold  2:  [0.89740000]
    fold  3:  [0.90220000]
    fold  4:  [0.90210000]
    ----
    MEAN:     [0.90300000] + [0.00345543]
    FULL:     [0.90300000]

model  1:     [LinearSVC]


/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


    fold  0:  [0.90200000]
    fold  1:  [0.90380000]
    fold  2:  [0.89500000]
    fold  3:  [0.90050000]
    fold  4:  [0.90030000]
    ----
    MEAN:     [0.90032000] + [0.00294170]
    FULL:     [0.90032000]

model  2:     [SGDClassifier]
    fold  0:  [0.89810000]
    fold  1:  [0.89290000]
    fold  2:  [0.89030000]
    fold  3:  [0.89190000]
    fold  4:  [0.89510000]
    ----
    MEAN:     [0.89366000] + [0.00271116]
    FULL:     [0.89366000]



In [15]:
import lightgbm

In [16]:
gbm = lightgbm.LGBMClassifier(random_state=42)
gbm.fit(S_train, y_train)
accuracy_score(y_test, gbm.predict(S_test))

0.884